In [1]:
from csv import reader
import tkinter as tk
from tkinter import ttk

In [2]:
with open('GRE_vocabulary.csv') as f:
    reader = reader(f)
    a=list(reader)
pristine_list=a[1:-1]
pristine_list.append(a[-1])

In [3]:
def refresh_listbox():
    unfamilar_words.delete(0,tk.END)
    for i in range(0,len(reshaped_list[m])):
        unfamilar_words.insert(0,"  "+reshaped_list[m][i][0]+" "+reshaped_list[m][i][1]+" "+reshaped_list[m][i][2])
    progress["text"]=str(bar["value"])+" / "+str(bar["maximum"])    

def show_question():
    global c
    global n
    if (c==1 or c==2):
        if (n >= len(reshaped_list[m])-1): # now that this list has finished its first traversal
            n=0
        else:
            n=n+1
        words_diaplay.delete(0,tk.END)
        words_diaplay.insert(0,reshaped_list[m][n][0])
        known_button['state'] = 'normal'
        unknown_button['state'] = 'normal'
        continue_button['state'] = 'disabled'
        
    c=3

def show_explain():
    words_diaplay.delete(0,tk.END)
    words_diaplay.insert(0,reshaped_list[m][n][0]+"   "+reshaped_list[m][n][1]+" （"+reshaped_list[m][n][2]+"）")    
    
def known():
    global c
    global reshaped_list
    if c==3:        
        show_explain()
        known_button['state'] = 'disabled'
        unknown_button['state'] = 'disabled'
        continue_button['state'] = 'normal'        
        reshaped_list[m].remove(reshaped_list[m][n])       
        refresh_listbox()
        if len(reshaped_list[m])==0:
            continue_button['state'] = 'disabled'
            unfamilar_words.delete(0,tk.END)
            unfamilar_words.insert(0,'You have finished this list, please switch to another one.')
            words_diaplay.delete(0,tk.END)
        else:
            refresh_listbox()
            bar["value"]=bar["value"]+1


    c=1
    
def unknown():
    global c
    if c==3:

        refresh_listbox()
        show_explain()
        known_button['state'] = 'disabled'
        unknown_button['state'] = 'disabled'
        continue_button['state'] = 'normal'
    c=2



def change_command(k):
    def change_groups():
        global n
        global m
        global c
        m=k
        show_question()
        words_diaplay.delete(0,tk.END)
        bar["maximum"]=len(reshaped_list[m])
        bar["value"]=1
        words_diaplay.insert(0,reshaped_list[m][0][0]) # initialize
        
        known_button['state'] = 'normal'
        unknown_button['state'] = 'normal'
        continue_button['state'] = 'disabled'        
        
        c=3
        refresh_listbox()

        label["text"]="List  "+str(m+1)
        
        n=0
    return change_groups

def eventhandler(event): # shortcut key settings
    if event.keysym == 'j':
        known_button.invoke()
    elif event.keysym == 'a':
        unknown_button.invoke()
    elif event.keysym == "z":
        continue_button.invoke()

In [4]:
root=tk.Tk() 
root.title("VerbalAid")
root.bind_all('<KeyPress>', eventhandler)

m=0
n=0
c=0 # adding 3-phase pointing variable to make it reliable

number_of_groups=20 # spliting the words into 20 groups
unit=int(len(pristine_list)/(number_of_groups*10))*10 

reshaped_list=list() # turning the initial 1*3000 data into 20*150
for i in range(0,number_of_groups):
    if (i==number_of_groups-1):
        reshaped_list.append(pristine_list[i*unit:-1])
        reshaped_list[i].append(pristine_list[-1])
    else:
        reshaped_list.append(pristine_list[i*unit:(i+1)*unit])
    

for k in range(0,number_of_groups):
    tk.Button(text=k+1,command=change_command(k),font = ('Helvetica', '14', 'normal')).grid(row=k,column=0,sticky=tk.W+tk.E+tk.N+tk.S,padx=3,pady=3)

words_diaplay=tk.Entry(root,font = ('Helvetica', '18', 'bold'),justify=tk.CENTER,state=tk.NORMAL)
words_diaplay.grid(column=1,row=1,columnspan=7,rowspan=2,sticky=tk.W+tk.E+tk.N+tk.S,padx=3,pady=3)

unfamilar_words  = tk.Listbox(root,selectmode = tk.EXTENDED,width=80)
unfamilar_words.grid(column=1,row=4,rowspan=17,columnspan=9,sticky=tk.W+tk.E+tk.N+tk.S,padx=3,pady=3)

known_button=tk.Button(text="Known (J)",command=known,font = ('Helvetica', '15', 'normal'),state='disabled')
known_button.grid(column=8,row=1,columnspan=2,sticky=tk.W+tk.E,padx=3,pady=3)

unknown_button=tk.Button(text="Unknown (A)",command=unknown,font = ('Helvetica', '15', 'normal'),state='disabled')
unknown_button.grid(column=8,row=2,columnspan=2,sticky=tk.W+tk.E,padx=3,pady=3)

continue_button=tk.Button(text="Continue (Z)",command=show_question,font = ('Helvetica', '15', 'normal'),state='disabled')
continue_button.grid(column=8,row=3,columnspan=2,sticky=tk.W+tk.E,padx=3,pady=3)

progress = tk.Label(root,text = ' ',font = ('Helvetica', '13', 'bold'))
progress.grid(column=7,row=3)

bar=ttk.Progressbar(root,orient = "horizontal", mode="determinate",maximum=unit)
bar.grid(column=1,row=3,columnspan=6,sticky=tk.W+tk.E+tk.N+tk.S,padx=3,pady=3)

scrl = tk.Scrollbar(root)
scrl.grid(row=4,column=10,rowspan=17,sticky=tk.W+tk.E+tk.N+tk.S,pady=3)
unfamilar_words.configure(yscrollcommand = scrl.set)
scrl['command'] = unfamilar_words.yview

label = tk.Label(root,text = ' ',font = ('Engravers MT', '15', 'bold'))
label.grid(column=5,row=0,sticky=tk.W+tk.E+tk.N+tk.S,padx=3,pady=3)

shl = ttk.Separator(root, orient=tk.HORIZONTAL)  
shl.grid(row=0,column=1,columnspan=4,sticky="we",padx=3,pady=3) 
shr = ttk.Separator(root, orient=tk.HORIZONTAL)  
shr.grid(row=0,column=6,columnspan=4,sticky="we",padx=3,pady=3) 

root.mainloop()